In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import layers, models, losses, optimizers
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn import preprocessing
import warnings
print(tf.config.list_physical_devices('GPU'))

: 

In [126]:
# import dataset
datapath = "../wildfire_dataset.nc"
wildfire_dataset = xr.open_dataset(datapath, engine="netcdf4")
# print(wildfire_dataset)
feature_list = wildfire_dataset.data_vars
feature_nums = len(feature_list)

# maybe predict more than one thing later, but for not only try to predict burned areas
remove_label = ["burned_areas", "ignition_points", "number_of_fires", "POP_DENS_2009", "POP_DENS_2010", "POP_DENS_2011", "POP_DENS_2012", "POP_DENS_2013", "POP_DENS_2014", "POP_DENS_2015", "POP_DENS_2016", "POP_DENS_2017", "POP_DENS_2018", "POP_DENS_2019", "POP_DENS_2020", "POP_DENS_2021", "ROAD_DISTANCE"]
X_label = [label for label in feature_list if label not in remove_label]
y_label = "burned_areas"

In [127]:
# take the first 5 time steps for all x and y to try creating a smaller dataset
num_samples = 10
timesteps_per_sample = 5
timestep_samples = num_samples*timesteps_per_sample
dataset_head = wildfire_dataset.head(indexers={"time": timestep_samples})
print(dataset_head)

dataset_X = dataset_head[X_label]
dataset_y = dataset_head[y_label]

# Create the y into a numpy matrix of shape (time, x, y)
dataset_y_np = dataset_y.to_numpy()
dataset_y_np = np.transpose(dataset_y_np, (0,2,1))

# Create the X into a numpy matrix of shape (time, x, y)
dataset_X_np = dataset_X[list(dataset_X.data_vars)[0]].to_numpy()
dataset_X_np = np.transpose(dataset_X_np, (0,2,1))
dataset_X_np = np.expand_dims(dataset_X_np, 3)

<xarray.Dataset>
Dimensions:             (time: 50, y: 983, x: 1253)
Coordinates:
  * time                (time) datetime64[ns] 2009-03-06T10:00:00 ... 2009-04...
  * x                   (x) float64 18.7 18.71 18.72 18.73 ... 28.88 28.89 28.9
  * y                   (y) float64 42.3 42.29 42.28 42.27 ... 34.32 34.31 34.3
    band                int64 ...
    spatial_ref         int64 ...
Data variables: (12/90)
    burned_areas        (time, y, x) float32 ...
    ignition_points     (time, y, x) float32 ...
    ndvi                (time, y, x) float32 ...
    number_of_fires     (time) int16 ...
    evi                 (time, y, x) float32 ...
    et                  (time, y, x) float32 ...
    ...                  ...
    POP_DENS_2016       (y, x) float32 ...
    POP_DENS_2017       (y, x) float32 ...
    POP_DENS_2018       (y, x) float32 ...
    POP_DENS_2019       (y, x) float32 ...
    POP_DENS_2020       (y, x) float32 ...
    POP_DENS_2021       (y, x) float32 ...
Attributes:


In [128]:
print("data vars:", type(dataset_X.data_vars))
# Takes each feature of the xarray Dataset and converts it into a DataArray 
# Also appends it into the new np array to make shape of (time x, y, features)
for index, feature in enumerate(list(dataset_X.data_vars)):
    if(index!=0):
        # Since wf_dataset_X_np is already initiaklized with the first element, skip
        new_np_arr = dataset_X[feature].to_numpy()
        if(len(new_np_arr.shape) == 2):
            # If a feature doesn't contain a time dimension (n), we extend the 2d matrix to 3d with copy of matrix n times
            # Might be able to use numpy broadcast instead
            new_np_arr = np.repeat(new_np_arr[:, :, np.newaxis], timestep_samples, axis=2)
            # Transpose feature to "time", "x", "y" format
            new_np_arr = np.transpose(new_np_arr)
        else:
            # Transpose feature to "time", "x", "y" format
            new_np_arr = np.transpose(new_np_arr, (0,2,1))
        if (np.isnan(new_np_arr).all()):
            # Precaution to alert if a feature has all NaN values
            warnings.warn(str(feature) + " feature's values are all NaNs")
        if (np.isnan(new_np_arr).any()):
            # Precaution to alert if a feature has all NaN values
            warnings.warn(str(feature) + " feature's values has NaNs")
        # new_np_arr = np.nan_to_num(new_np_arr)
        # print(new_np_arr)
        dataset_X_np = np.concatenate((dataset_X_np, np.expand_dims(new_np_arr, axis=3)), axis=3)
    print(feature)
    print(dataset_X_np.shape)

data vars: <class 'xarray.core.dataset.DataVariables'>
ndvi
(50, 1253, 983, 1)


/tmp/ipykernel_10715/64375616.py:22: UserWarning: evi feature's values has NaNs
  warnings.warn(str(feature) + " feature's values has NaNs")


evi
(50, 1253, 983, 2)
et
(50, 1253, 983, 3)


/tmp/ipykernel_10715/64375616.py:22: UserWarning: lst_day feature's values has NaNs
  warnings.warn(str(feature) + " feature's values has NaNs")


lst_day
(50, 1253, 983, 4)


...

min_rh
(50, 1253, 983, 34)


/tmp/ipykernel_10715/64375616.py:22: UserWarning: avg_rh feature's values has NaNs
  warnings.warn(str(feature) + " feature's values has NaNs")


avg_rh
(50, 1253, 983, 35)
CLC_2006
(50, 1253, 983, 36)
CLC_2006_0
(50, 1253, 983, 37)
CLC_2006_1
(50, 1253, 983, 38)
CLC_2006_2
(50, 1253, 983, 39)
CLC_2006_3
(50, 1253, 983, 40)
CLC_2006_4
(50, 1253, 983, 41)
CLC_2006_5
(50, 1253, 983, 42)
CLC_2006_6
(50, 1253, 983, 43)
CLC_2006_7
(50, 1253, 983, 44)
CLC_2006_8
(50, 1253, 983, 45)
CLC_2006_9
(50, 1253, 983, 46)
CLC_2012
(50, 1253, 983, 47)
CLC_2012_0
(50, 1253, 983, 48)
CLC_2012_1
(50, 1253, 983, 49)
CLC_2012_2
(50, 1253, 983, 50)
CLC_2012_3
(50, 1253, 983, 51)
CLC_2012_4
(50, 1253, 983, 52)
CLC_2012_5
(50, 1253, 983, 53)
CLC_2012_6
(50, 1253, 983, 54)
CLC_2012_7
(50, 1253, 983, 55)
CLC_2012_8
(50, 1253, 983, 56)
CLC_2012_9
(50, 1253, 983, 57)
CLC_2018
(50, 1253, 983, 58)
CLC_2018_0
(50, 1253, 983, 59)
CLC_2018_1
(50, 1253, 983, 60)
CLC_2018_2
(50, 1253, 983, 61)
CLC_2018_3
(50, 1253, 983, 62)
CLC_2018_4
(50, 1253, 983, 63)
CLC_2018_5
(50, 1253, 983, 64)
CLC_2018_6
(50, 1253, 983, 65)
CLC_2018_7
(50, 1253, 983, 66)
CLC_2018_8
(50, 12

/tmp/ipykernel_10715/64375616.py:22: UserWarning: ASPECT feature's values has NaNs
  warnings.warn(str(feature) + " feature's values has NaNs")


ASPECT
(50, 1253, 983, 71)
ROUGHNESS
(50, 1253, 983, 72)
WATERWAY_DISTANCE
(50, 1253, 983, 73)


In [129]:
# rearrange so that it matches what keras expects (time, features, x, y) instead of (time, x, y, features)
# dataset_X_np = np.moveaxis(dataset_X_np, 3, 1)
print("dataset_X_np.shape", dataset_X_np.shape)
print("dataset_y_np.shape", dataset_y_np.shape)

dataset_X_np.shape (50, 1253, 983, 73)
dataset_y_np.shape (50, 1253, 983)


In [130]:
if 1 in dataset_y_np:
    print("Fire exists") 
print("Output classes: ", np.unique(dataset_y_np))
# class_weights = class_weight.compute_class_weight(class_weight = "balanced", classes = np.unique(wf_dataset_y_np), y = wf_dataset_y_np)
# print(class_weights)

Output classes:  [0.]


In [131]:
# Create samples (samples, time, features, x, y)
# NOT IMPLEMENTED: Each samples are 4 days with 2 day overlap between each one
dataset_X_np = np.expand_dims(dataset_X_np, axis=0)
print("wf_dataset_X_np:", type(dataset_X_np))
print("wf_dataset_X_np(expand dims):", dataset_X_np)
dataset_X_np = np.reshape(dataset_X_np, (num_samples, timesteps_per_sample, dataset_X_np.shape[2], dataset_X_np.shape[3], dataset_X_np.shape[4]))
print("dataset_X_np(reshape):", type(dataset_X_np))

print("dataset_X_np.shape: ", dataset_X_np.shape)

wf_dataset_X_np: <class 'numpy.ndarray'>
wf_dataset_X_np(expand dims): [[[[[   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    ...
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]]

   [[   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    ...
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]]

   [[   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    ...
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    nan 3276.6 ...    nan    0.     0. ]
    [   nan    na

In [132]:
# Deal with y labels
replacement_y_np = np.zeros(timestep_samples)
for i in range(dataset_y_np.shape[0]):
    if 1 in dataset_y_np[i]:
        replacement_y_np[i] = 1
# print(replacement_y_np)
# print(replacement_y_np.shape)

replacement_y_np = np.reshape(replacement_y_np, (num_samples, timesteps_per_sample))
# print(replacement_y_np)
# print(replacement_y_np.shape)

In [133]:
# train test split (70/30 split)
# split along axis 0
dataset_X_np_split = np.split(dataset_X_np, [7, 10])
dataset_y_np_split = np.split(replacement_y_np, [7, 10])
print("dataset_X_np_split:", type(dataset_X_np_split))
X_train = dataset_X_np_split[0]
X_test = dataset_X_np_split[1]
y_train = dataset_y_np_split[0]
y_test = dataset_y_np_split[1]

dataset_X_np_split: <class 'list'>


In [134]:
print(y_train.shape)

(7, 5)


In [135]:
# NEW Normalize X_train and X_test
#(samples, time, channels, rows, cols)
# Loop through each feature
for i in range(X_train.shape[1]):
    print("X_train[:,i,:,;].shape: ", X_train[:,:,:,:,i].shape)
    print("X_test[:,i,:,;].shape: ", X_test[:,:,:,:,i].shape)
    # Standard Scaler
    sc = StandardScaler()
    # Every X_train/test feature will be reshaped to a 2d array
    X_train_2d = X_train[:,:,:,:,i].reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2]*X_train.shape[3])
    X_test_2d = X_test[:,:,:,:,i].reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2]*X_test.shape[3])
    # Normalize
    X_train_transformed = sc.fit_transform(X_train_2d)
    X_test_transformed = sc.transform(X_test_2d)
    # Reshape to 4d (samples, time, x, y)
    X_train_transformed = X_train_transformed.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3])
    X_test_transformed = X_test_transformed.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], X_test.shape[3])
    # Store normalized feature in X_train
    X_train[:,:,:,:,i] = X_train_transformed
    X_test[:,:,:,:,i] = X_test_transformed


X_train[:,i,:,;].shape:  (7, 5, 1253, 983)
X_test[:,i,:,;].shape:  (3, 5, 1253, 983)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)


X_train[:,i,:,;].shape:  (7, 5, 1253, 983)
X_test[:,i,:,;].shape:  (3, 5, 1253, 983)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)


X_train[:,i,:,;].shape:  (7, 5, 1253, 983)
X_test[:,i,:,;].shape:  (3, 5, 1253, 983)
X_train[:,i,:,;].shape:  (7, 5, 1253, 983)
X_test[:,i,:,;].shape:  (3, 5, 1253, 983)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)


X_train[:,i,:,;].shape:  (7, 5, 1253, 983)
X_test[:,i,:,;].shape:  (3, 5, 1253, 983)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.8/dist-packages/sklearn/utils/extmath.py:687: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs, dtype=np.float64)


In [136]:
# # Normalize X_train and X_test
# #(samples, time, channels, rows, cols)
# # Loop through each feature
# for i in range(X_train.shape[1]):
#     print("X_train[:,i,:,;].shape: ", X_train[:,:,i,:,:].shape)
#     print("X_test[:,i,:,;].shape: ", X_test[:,:,i,:,:].shape)
#     # Standard Scaler
#     sc = StandardScaler()
#     # Every X_train/test feature will be reshaped to a 2d array
#     X_train_2d = X_train[:,:,i,:,:].reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[3]*X_train.shape[4])
#     X_test_2d = X_test[:,:,i,:,:].reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[3]*X_test.shape[4])
#     # Normalize
#     X_train_transformed = sc.fit_transform(X_train_2d)
#     X_test_transformed = sc.transform(X_test_2d)
#     # Reshape to 4d (samples, time, x, y)
#     X_train_transformed = X_train_transformed.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[3], X_train.shape[4])
#     X_test_transformed = X_test_transformed.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[3], X_test.shape[4])
#     # Store normalized feature in X_train
#     X_train[:,:,i,:,:] = X_train_transformed
#     X_test[:,:,i,:,:] = X_test_transformed


In [137]:
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print("input shape: ", X_train.shape[-4:])

X_train:  (7, 5, 1253, 983, 73)
X_test:  (3, 5, 1253, 983, 73)
y_train:  (7, 5)
y_test:  (3, 5)
input shape:  (5, 1253, 983, 73)


In [138]:
# # revert from (time, features, x, y) to (time, x, y, features) for data_format="channels_last"
# X_train = np.moveaxis(X_train, 2, -1)
# X_test = np.moveaxis(X_test, 2, -1)
# print("X_train.shape", X_train.shape)
# print("X_test.shape", X_test.shape)

In [139]:
def build_ConvLSTM():
    convlstm = models.Sequential()
    convlstm.add(layers.Input(shape=X_train.shape[-4:]))
    convlstm.add(layers.ConvLSTM2D(filters=256, kernel_size=(5,5), data_format="channels_last", return_sequences=True))
    convlstm.add(layers.BatchNormalization())
    convlstm.add(layers.ConvLSTM2D(filters=128, kernel_size=(3,3), data_format="channels_last", return_sequences=True))
    convlstm.add(layers.BatchNormalization())
    convlstm.add(layers.ConvLSTM2D(filters=64, kernel_size=(2,2), data_format="channels_last", return_sequences=True))
    convlstm.add(layers.BatchNormalization())
    convlstm.add(layers.ConvLSTM2D(filters=32, kernel_size=(1,1), data_format="channels_last", return_sequences=True))
    convlstm.add(layers.Conv3D(filters=1, kernel_size=(1, 1246, 976), data_format="channels_last", activation="sigmoid"))
    # kernel_size=(timesteps_per_sample, 80, 1246)
    convlstm.compile(
        loss=losses.binary_crossentropy, optimizer=optimizers.Adam(), metrics=[tf.keras.metrics.Accuracy()]
    )
    return convlstm

In [140]:
model = build_ConvLSTM()
print(model.summary())
epochs = 10
batch_size = 1
y_train_test = np.expand_dims(y_train, axis=(2,3,4))
print(y_train_test.shape)
history = model.fit(X_train, y_train_test, epochs=epochs, batch_size=batch_size, verbose=True)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_48 (ConvLSTM2D)  (None, 5, 1249, 979, 256  8423424  
                             )                                   
                                                                 
 batch_normalization_36 (Bat  (None, 5, 1249, 979, 256  1024     
 chNormalization)            )                                   
                                                                 
 conv_lstm2d_49 (ConvLSTM2D)  (None, 5, 1247, 977, 128  1769984  
                             )                                   
                                                                 
 batch_normalization_37 (Bat  (None, 5, 1247, 977, 128  512      
 chNormalization)            )                                   
                                                                 
 conv_lstm2d_50 (ConvLSTM2D)  (None, 5, 1246, 976, 64

2023-05-03 15:08:59.243665: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:1014] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'Func/gradient_tape/sequential_12/conv_lstm2d_51/while/sequential_12/conv_lstm2d_51/while_grad/body/_743/input/_2035' -> 'gradient_tape/sequential_12/conv_lstm2d_51/while/sequential_12/conv_lstm2d_51/while_grad/body/_743/gradient_tape/sequential_12/conv_lstm2d_51/while/gradients/AddN', 'Func/gradient_tape/sequential_12/conv_lstm2d_50/while/sequential_12/conv_lstm2d_50/while_grad/body/_938/input/_2154' -> 'gradient_tape/sequential_12/conv_lstm2d_50/while/sequential_12/conv_lstm2d_50/while_grad/body/_938/gradient_tape/sequential_12/conv_lstm2d_50/while/gradients/AddN', 'Func/gradient_tape/sequential_12/conv_lstm2d_49/while/sequential_12/conv_lstm2d_49/while_grad/body/_1133/input/_2273' -> 'gradient_tape/sequential_12/conv_lstm2d_49/while/sequential_12/conv_lstm2d_49/while_grad/body/_1133/g

2023-05-03 15:10:02.902376: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1252117504)requested by op sequential_12/conv_lstm2d_48/while/body/_1/sequential_12/conv_lstm2d_48/while/convolution
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-03 15:10:02.902426: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-03 15:10:02.902443: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 189, Chunks in use: 189. 47.2KiB allocated for chunks. 47.2KiB in use in bin. 13.4KiB client-requested in use in bin.
2023-05-03 15:10:02.902451: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.5KiB allocated for chunks. 24.5KiB 

2023-05-03 15:10:12.906831: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1252117504)requested by op sequential_12/conv_lstm2d_48/while/body/_1/sequential_12/conv_lstm2d_48/while/convolution_3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-03 15:10:12.906885: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-03 15:10:12.906905: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 189, Chunks in use: 189. 47.2KiB allocated for chunks. 47.2KiB in use in bin. 13.4KiB client-requested in use in bin.
2023-05-03 15:10:12.906921: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.5KiB allocated for chunks. 24.5Ki

2023-05-03 15:10:22.911254: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1252117504)requested by op sequential_12/conv_lstm2d_48/while/body/_1/sequential_12/conv_lstm2d_48/while/convolution_4
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-03 15:10:22.911297: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-03 15:10:22.911308: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 189, Chunks in use: 189. 47.2KiB allocated for chunks. 47.2KiB in use in bin. 13.4KiB client-requested in use in bin.
2023-05-03 15:10:22.911316: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.5KiB allocated for chunks. 24.5Ki

2023-05-03 15:10:32.916972: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1252117504)requested by op sequential_12/conv_lstm2d_48/while/body/_1/sequential_12/conv_lstm2d_48/while/convolution_5
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-03 15:10:32.917018: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-03 15:10:32.917035: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 189, Chunks in use: 189. 47.2KiB allocated for chunks. 47.2KiB in use in bin. 13.4KiB client-requested in use in bin.
2023-05-03 15:10:32.917044: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.5KiB allocated for chunks. 24.5Ki

2023-05-03 15:10:42.921954: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1252117504)requested by op sequential_12/conv_lstm2d_48/while/body/_1/sequential_12/conv_lstm2d_48/while/convolution_6
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-03 15:10:42.922000: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-03 15:10:42.922011: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 189, Chunks in use: 189. 47.2KiB allocated for chunks. 47.2KiB in use in bin. 13.4KiB client-requested in use in bin.
2023-05-03 15:10:42.922019: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.5KiB allocated for chunks. 24.5Ki

2023-05-03 15:10:52.926302: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1252117504)requested by op sequential_12/conv_lstm2d_48/while/body/_1/sequential_12/conv_lstm2d_48/while/convolution_7
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-03 15:10:52.926354: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-03 15:10:52.926366: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 189, Chunks in use: 189. 47.2KiB allocated for chunks. 47.2KiB in use in bin. 13.4KiB client-requested in use in bin.
2023-05-03 15:10:52.926374: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 48, Chunks in use: 48. 24.5KiB allocated for chunks. 24.5Ki

KeyboardInterrupt: 